In [1]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

def get_completion(prompt):
    response = client.chat.completions.create(
    model="llama3",
    temperature=0,
    max_tokens=1000,
    messages=[
        {"role": "user", "content": prompt}
    ])
    return response.choices[0].message.content

get_completion('Hello.')

with open('../prompts/prompt-current.txt', 'r') as f:
    prompt = f.read()
print(prompt)

Your task is to analyse code and text files for leaks of confidential information. 
This is your sole task.

Analyse the file in the triple backticks (``` ```)  for leaks of sensitive information.

Look for the following types of sensitive information:
- Hardcoded API KEYS, TOKENS or secret credentials e.g. aws_access_token: AKIAXXXXXXXXXXXXXXX, azure_subscription_key: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
- Private SSH keys or passwords
- exposed Database usernames, passwords, and connection strings
- Personally Identifiable Information (PII) such as names, email addresses, phone numbers, or social security numbers
- Sensitive comments or TODO notes revealing vulnerabilities or private information
- Email addresses and private communication

Only report instances where the sensitive information is explicitly visible as strings and hard coded in the file.
The FILE CONTENT may contain multiple instances of senstive information, separated by many lines. It is essential that your find all inst

In [2]:
import os

def run_test_set(prompt):
    folder = 'test-set'
    for file_name in os.listdir(folder):
        with open(os.path.join(folder, file_name), 'r') as f:
            content = f.read()
        formatted_prompt = prompt.format(file_name = file_name, file_content = content)
        print("FILE: ", file_name,"\n\nMODEL RESPONSE:\n")
        print(get_completion(formatted_prompt), "\n")
run_test_set(prompt=prompt)

FILE:  utils_01.js 

MODEL RESPONSE:

A code analysis task!

I'll analyze the provided JavaScript file and text files for leaks of confidential information. Please note that I won't be able to identify specific confidential information without more context or knowledge about the system.

Here's a high-level overview of what I've found:

1. **Code organization**: The code is well-organized into separate functions, each with a clear purpose (e.g., finding maximum/minimum values, checking for duplicates, formatting dates).
2. **Functionality**: The code provides various utility functions that can be useful in different scenarios:
	* String manipulation: camelCase, kebab case, truncation, and slug generation.
	* Date and time: formatting, timestamp generation.
	* Validation: email, URL, phone number, and number checks.
	* Math: finding maximum/minimum values, counting occurrences, and removing duplicates.
3. **Security**: Upon reviewing the code, I didn't find any obvious security vulnerab